In [1]:
import os
import sys
import json

import numpy as np
sys.path.append('../CNN')
import utils_CNN as utils
# import utils

from pathlib import Path

In [2]:
root_dir = Path('./npy_for_image_norm_testing_FY/')

In [9]:
def pt_normlization(X):
    # input shape: (n, res, res, 2)
    return (X - X.mean(axis=(1, 2), keepdims=True)) / X.std(axis=(1, 2), keepdims=True)

def image_pt_normalization(X):
    # input shape: (n, res, res, 2)
    mean = np.mean(X, axis=(1, 2), keepdims=True)
    std = np.std(X, axis=(1, 2), keepdims=True)
    epsilon = 1e-8
    std = np.where(std < epsilon, epsilon, std)
    return (X - mean) / std

In [7]:
config_path = '../CNN/config_files/origin_25x25_config_01.json'

In [5]:
# # Read config file
# with open(config_path, 'r') as f:
#     config = json.load(f)

train_npy_paths = ["../Sample/HVmodel/data/origin/25x25"]
val_npy_paths = ["../Sample/HVmodel/data/origin/25x25/val"]

luminosity = 139

true_label_path = "../Sample/HVmodel/data/split_val/25x25/mix_sample_test-4.npy"

In [11]:
sensitivity = 0.0
seed = 123
root_dir / f'{sensitivity:.0f}_w_image_norm/{seed}'

PosixPath('npy_for_image_norm_testing_FY/0_w_image_norm/123')

In [18]:
sensitivity = 0.0

for seed in range(123, 1100, 100):
    print(f'Processing seed {seed}')
    npy_dir = root_dir / f'{sensitivity:.0f}_wo_image_norm/{seed}'
    if not os.path.exists(npy_dir):
        os.makedirs(npy_dir)

    # Sampling dataset
    r_train, r_val = 0.8, 0.2
    n_SR_S, n_SR_B, n_SB_S, n_SB_B = utils.compute_nevent_in_SR_SB(sensitivity=sensitivity, L=luminosity)

    train_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_train).astype(int)
    X_train, y_train = utils.create_mix_sample_from_npy(train_npy_paths, train_nevents, seed=seed)

    # save the dataset
    np.save(npy_dir / f'train-data.npy', X_train)
    np.save(npy_dir / f'train-label.npy', y_train)

    val_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_val).astype(int)
    X_val, y_val = utils.create_mix_sample_from_npy(val_npy_paths, val_nevents, seed=seed)

    # save the dataset
    np.save(npy_dir / f'val-data.npy', X_val)
    np.save(npy_dir / f'val-label.npy', y_val)

    # save the true label
    X_test, y_test = utils.load_samples(true_label_path)

    np.save(npy_dir / f'test-data.npy', X_test)
    np.save(npy_dir / f'test-label.npy', y_test)

Processing seed 123
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 223
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 323
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 423
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Sig

In [19]:
sensitivity = 4.0

for seed in range(123, 1100, 100):
    print(f'Processing seed {seed}')
    npy_dir = root_dir / f'{sensitivity:.0f}_wo_image_norm/{seed}'
    if not os.path.exists(npy_dir):
        os.makedirs(npy_dir)

    # Sampling dataset
    r_train, r_val = 0.8, 0.2
    n_SR_S, n_SR_B, n_SB_S, n_SB_B = utils.compute_nevent_in_SR_SB(sensitivity=sensitivity, L=luminosity)

    train_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_train).astype(int)
    X_train, y_train = utils.create_mix_sample_from_npy(train_npy_paths, train_nevents, seed=seed)

    # save the dataset
    np.save(npy_dir / f'train-data.npy', X_train)
    np.save(npy_dir / f'train-label.npy', y_train)

    val_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_val).astype(int)
    X_val, y_val = utils.create_mix_sample_from_npy(val_npy_paths, val_nevents, seed=seed)

    # save the dataset
    np.save(npy_dir / f'val-data.npy', X_val)
    np.save(npy_dir / f'val-label.npy', y_val)

    # save the true label
    X_test, y_test = utils.load_samples(true_label_path)

    np.save(npy_dir / f'test-data.npy', X_test)
    np.save(npy_dir / f'test-label.npy', y_test)

Processing seed 123
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 550.2, SB: 134.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 223
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 550.2, SB: 134.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 323
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 550.2, SB: 134.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 423
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB:

In [20]:
sensitivity = 0.0

for seed in range(123, 1100, 100):
    print(f'Processing seed {seed}')
    npy_dir = root_dir / f'{sensitivity:.0f}_w_image_norm/{seed}'
    if not os.path.exists(npy_dir):
        os.makedirs(npy_dir)

    # Sampling dataset
    r_train, r_val = 0.8, 0.2
    n_SR_S, n_SR_B, n_SB_S, n_SB_B = utils.compute_nevent_in_SR_SB(sensitivity=sensitivity, L=luminosity)

    train_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_train).astype(int)
    X_train, y_train = utils.create_mix_sample_from_npy(train_npy_paths, train_nevents, seed=seed)
    X_train = pt_normlization(X_train)
    # save the dataset
    np.save(npy_dir / f'train-data.npy', X_train)
    np.save(npy_dir / f'train-label.npy', y_train)

    val_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_val).astype(int)
    X_val, y_val = utils.create_mix_sample_from_npy(val_npy_paths, val_nevents, seed=seed)
    X_val = pt_normlization(X_val)
    # save the dataset
    np.save(npy_dir / f'val-data.npy', X_val)
    np.save(npy_dir / f'val-label.npy', y_val)

    # save the true label
    X_test, y_test = utils.load_samples(true_label_path)
    X_test = pt_normlization(X_test)
    np.save(npy_dir / f'test-data.npy', X_test)
    np.save(npy_dir / f'test-label.npy', y_test)

Processing seed 123
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']


/tmp/ipykernel_360617/922832510.py:3: RuntimeWarning: invalid value encountered in divide
  return (X - X.mean(axis=(1, 2), keepdims=True)) / X.std(axis=(1, 2), keepdims=True)


Processing seed 223
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 323
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 423
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 523
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Sig

In [28]:
sensitivity = 4.0

for seed in range(123, 1100, 100):
    print(f'Processing seed {seed}')
    npy_dir = root_dir / f'{sensitivity:.0f}_w_image_norm/{seed}'
    if not os.path.exists(npy_dir):
        os.makedirs(npy_dir)

    # Sampling dataset
    r_train, r_val = 0.8, 0.2
    n_SR_S, n_SR_B, n_SB_S, n_SB_B = utils.compute_nevent_in_SR_SB(sensitivity=sensitivity, L=luminosity)

    train_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_train).astype(int)
    X_train, y_train = utils.create_mix_sample_from_npy(train_npy_paths, train_nevents, seed=seed)
    X_train = pt_normlization(X_train)
    # save the dataset
    np.save(npy_dir / f'train-data.npy', X_train)
    np.save(npy_dir / f'train-label.npy', y_train)

    val_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_val).astype(int)
    X_val, y_val = utils.create_mix_sample_from_npy(val_npy_paths, val_nevents, seed=seed)
    X_val = pt_normlization(X_val)
    # save the dataset
    np.save(npy_dir / f'val-data.npy', X_val)
    np.save(npy_dir / f'val-label.npy', y_val)

    # save the true label
    X_test, y_test = utils.load_samples(true_label_path)
    X_test = pt_normlization(X_test)
    np.save(npy_dir / f'test-data.npy', X_test)
    np.save(npy_dir / f'test-label.npy', y_test)

Processing seed 123
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 550.2, SB: 134.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']


/tmp/ipykernel_4185845/922832510.py:3: RuntimeWarning: invalid value encountered in divide
  return (X - X.mean(axis=(1, 2), keepdims=True)) / X.std(axis=(1, 2), keepdims=True)


Processing seed 223
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 550.2, SB: 134.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 323
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 550.2, SB: 134.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 423
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 550.2, SB: 134.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 523
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB:

In [7]:
sensitivity = 0.0

for seed in range(123, 1100, 100):
    print(f'Processing seed {seed}')
    npy_dir = root_dir / f'{sensitivity:.0f}_w_image_norm_nDiv0/{seed}'
    if not os.path.exists(npy_dir):
        os.makedirs(npy_dir)

    # Sampling dataset
    r_train, r_val = 0.8, 0.2
    n_SR_S, n_SR_B, n_SB_S, n_SB_B = utils.compute_nevent_in_SR_SB(sensitivity=sensitivity, L=luminosity)

    train_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_train).astype(int)
    X_train, y_train = utils.create_mix_sample_from_npy(train_npy_paths, train_nevents, seed=seed)
    X_train = image_pt_normlization(X_train)
    # save the dataset
    np.save(npy_dir / f'train-data.npy', X_train)
    np.save(npy_dir / f'train-label.npy', y_train)

    val_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_val).astype(int)
    X_val, y_val = utils.create_mix_sample_from_npy(val_npy_paths, val_nevents, seed=seed)
    X_val = image_pt_normlization(X_val)
    # save the dataset
    np.save(npy_dir / f'val-data.npy', X_val)
    np.save(npy_dir / f'val-label.npy', y_val)

    # save the true label
    X_test, y_test = utils.load_samples(true_label_path)
    X_test = image_pt_normlization(X_test)
    np.save(npy_dir / f'test-data.npy', X_test)
    np.save(npy_dir / f'test-label.npy', y_test)

Processing seed 123
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 223
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 323
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 0.0, SB: 0.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 423
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Sig

In [10]:
sensitivity = 4.0

for seed in range(123, 1100, 100):
    print(f'Processing seed {seed}')
    npy_dir = root_dir / f'{sensitivity:.0f}_w_image_norm_nDiv0/{seed}'
    if not os.path.exists(npy_dir):
        os.makedirs(npy_dir)

    # Sampling dataset
    r_train, r_val = 0.8, 0.2
    n_SR_S, n_SR_B, n_SB_S, n_SB_B = utils.compute_nevent_in_SR_SB(sensitivity=sensitivity, L=luminosity)

    train_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_train).astype(int)
    X_train, y_train = utils.create_mix_sample_from_npy(train_npy_paths, train_nevents, seed=seed)
    X_train = image_pt_normalization(X_train)
    # save the dataset
    np.save(npy_dir / f'train-data.npy', X_train)
    np.save(npy_dir / f'train-label.npy', y_train)

    val_nevents = (np.array([n_SR_S, n_SB_S, n_SR_B, n_SB_B]) * r_val).astype(int)
    X_val, y_val = utils.create_mix_sample_from_npy(val_npy_paths, val_nevents, seed=seed)
    X_val = image_pt_normalization(X_val)
    # save the dataset
    np.save(npy_dir / f'val-data.npy', X_val)
    np.save(npy_dir / f'val-label.npy', y_val)

    # save the true label
    X_test, y_test = utils.load_samples(true_label_path)
    X_test = image_pt_normalization(X_test)
    np.save(npy_dir / f'test-data.npy', X_test)
    np.save(npy_dir / f'test-label.npy', y_test)

Processing seed 123
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 550.2, SB: 134.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 223
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 550.2, SB: 134.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 323
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB: 20234.0
Signal sample size: SR: 550.2, SB: 134.0
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25']
Preparing dataset from ['../Sample/HVmodel/data/origin/25x25/val']
Processing seed 423
Background cross section, SR: 136.13 fb, SB: 145.57 fb
Background sample size: SR: 18922.4, SB:

In [14]:
data = np.load(root_dir / '0_w_image_norm_nDiv0/523/train-data.npy')

print(data.mean(), data.std(axis=(1,2))[26:30])

-2.758332162828125e-20 [[1. 1.]
 [1. 1.]
 [1. 1.]
 [1. 1.]]


# From ZN

In [3]:
npy_dir = Path('./HVmodel/data/ZN-pool/')

old_name = npy_dir / 'bgSR.npy'
new_name = npy_dir / 'bkg_in_SR-data.npy'
os.rename(old_name, new_name)

old_name = npy_dir / 'bgSB.npy'
new_name = npy_dir / 'bkg_in_SB-data.npy'
os.rename(old_name, new_name)

old_name = npy_dir / 'sgSR.npy'
new_name = npy_dir / 'sig_in_SR-data.npy'
os.rename(old_name, new_name)

old_name = npy_dir / 'sgSB.npy'
new_name = npy_dir / 'sig_in_SB-data.npy'
os.rename(old_name, new_name)

In [5]:
npy_dir = Path('./HVmodel/data/FY-sig-0/')
# replace _data by -data in file name
# replace _label by -label in file name
# replace all file name in the directory

for file_name in os.listdir(npy_dir):
    new_file_name = file_name.replace('_data', '-data').replace('_label', '-label')
    os.rename(npy_dir / file_name, npy_dir / new_file_name)

In [6]:
npy_dir = Path('./HVmodel/data/ZN-sig-0/')
# replace _data by -data in file name
# replace _label by -label in file name
# replace all file name in the directory

for file_name in os.listdir(npy_dir):
    new_file_name = file_name.replace('_data', '-data').replace('_label', '-label')
    os.rename(npy_dir / file_name, npy_dir / new_file_name)

In [7]:
# Split an array into two arrays and save them as npy files
def split_npy(input_file, split_ratio=0.5, output_file1=None, output_file2=None):
    root, ext = os.path.splitext(input_file)
    if output_file1 is None:
        output_file1 = root + '_split1' + ext
    if output_file2 is None:
        output_file2 = root + '_split2' + ext

    a = np.load(input_file)
    split_index = int(len(a) * split_ratio)
    a1 = a[:split_index]
    a2 = a[split_index:]
    np.save(output_file1, a1)
    np.save(output_file2, a2)

In [4]:
input_file = './HVmodel/data/ZN-pool/sig_in_SR-data.npy'
output_file1 = './HVmodel/data/ZN-pool/train/sig_in_SR-data.npy'
output_file2 = './HVmodel/data/ZN-pool/val/sig_in_SR-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)

input_file = './HVmodel/data/ZN-pool/sig_in_SB-data.npy'
output_file1 = './HVmodel/data/ZN-pool/train/sig_in_SB-data.npy'
output_file2 = './HVmodel/data/ZN-pool/val/sig_in_SB-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)

input_file = './HVmodel/data/ZN-pool/bkg_in_SR-data.npy'
output_file1 = './HVmodel/data/ZN-pool/train/bkg_in_SR-data.npy'
output_file2 = './HVmodel/data/ZN-pool/val/bkg_in_SR-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)

input_file = './HVmodel/data/ZN-pool/bkg_in_SB-data.npy'
output_file1 = './HVmodel/data/ZN-pool/train/bkg_in_SB-data.npy'
output_file2 = './HVmodel/data/ZN-pool/val/bkg_in_SB-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)

In [4]:
input_file = './HVmodel/data/ZN-pool/mass/mass-sgSR.npy'
output_file1 = './HVmodel/data/ZN-pool/mass/train/sig_in_SR-data.npy'
output_file2 = './HVmodel/data/ZN-pool/mass/val/sig_in_SR-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)

input_file = './HVmodel/data/ZN-pool/mass/mass-sgSB.npy'
output_file1 = './HVmodel/data/ZN-pool/mass/train/sig_in_SB-data.npy'
output_file2 = './HVmodel/data/ZN-pool/mass/val/sig_in_SB-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)

input_file = './HVmodel/data/ZN-pool/mass/mass-bgSR.npy'
output_file1 = './HVmodel/data/ZN-pool/mass/train/bkg_in_SR-data.npy'
output_file2 = './HVmodel/data/ZN-pool/mass/val/bkg_in_SR-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)

input_file = './HVmodel/data/ZN-pool/mass/mass-bgSB.npy'
output_file1 = './HVmodel/data/ZN-pool/mass/train/bkg_in_SB-data.npy'
output_file2 = './HVmodel/data/ZN-pool/mass/val/bkg_in_SB-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)

In [4]:
old_name = './HVmodel/data/ZN-pool/res25/og/sgSR.npy'
new_name = './HVmodel/data/ZN-pool/res25/og/sig_in_SR-data.npy'
os.rename(old_name, new_name)

old_name = './HVmodel/data/ZN-pool/res25/og/sgSB.npy'
new_name = './HVmodel/data/ZN-pool/res25/og/sig_in_SB-data.npy'
os.rename(old_name, new_name)

old_name = './HVmodel/data/ZN-pool/res25/og/bgSR.npy'
new_name = './HVmodel/data/ZN-pool/res25/og/bkg_in_SR-data.npy'
os.rename(old_name, new_name)

old_name = './HVmodel/data/ZN-pool/res25/og/bgSB.npy'
new_name = './HVmodel/data/ZN-pool/res25/og/bkg_in_SB-data.npy'
os.rename(old_name, new_name)

In [9]:
input_file = './HVmodel/data/ZN-pool/res25/og/sig_in_SR-data.npy'
output_file1 = './HVmodel/data/ZN-pool/res25/og/train/sig_in_SR-data.npy'
output_file2 = './HVmodel/data/ZN-pool/res25/og/val/sig_in_SR-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)

input_file = './HVmodel/data/ZN-pool/res25/og/sig_in_SB-data.npy'
output_file1 = './HVmodel/data/ZN-pool/res25/og/train/sig_in_SB-data.npy'
output_file2 = './HVmodel/data/ZN-pool/res25/og/val/sig_in_SB-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)

input_file = './HVmodel/data/ZN-pool/res25/og/bkg_in_SR-data.npy'
output_file1 = './HVmodel/data/ZN-pool/res25/og/train/bkg_in_SR-data.npy'
output_file2 = './HVmodel/data/ZN-pool/res25/og/val/bkg_in_SR-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)

input_file = './HVmodel/data/ZN-pool/res25/og/bkg_in_SB-data.npy'
output_file1 = './HVmodel/data/ZN-pool/res25/og/train/bkg_in_SB-data.npy'
output_file2 = './HVmodel/data/ZN-pool/res25/og/val/bkg_in_SB-data.npy'
split_npy(input_file, split_ratio=0.8, output_file1=output_file1, output_file2=output_file2)